In [102]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
df = pd.read_excel("ex1.xlsx", sheet_name="exercises")
df.columns = df.iloc[0] # load first row as column names
df = df.drop(df.index[0]) # drop first row

# magic row: minimum training time for each exercise
last_row = df.iloc[-1]
assert last_row["Name"] == "Minimum Weekly Workout Time"
display(df.head())

df.columns

,Name,Category,Sets,Set time in min,Break time between sets in min,Total time,Priority,NaN,Biceps,Chest,Triceps,Shoulder,Abdominals,Upper Back,Lower Back,Upper Thighs,Lower Thighs,Calves,Glutes
1,Cycling Warmup,warm up,1,7,0,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN
2,Rowing Warmup,warm up,1,7,0,7,4,NaN,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,Running Warmup,warm up,1,7,0,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Abduction Machine,resistance,3,1,1.5,6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,0
5,Adduction Machine,resistance,3,1,1.5,6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,2


Index([                          'Name',                       'Category',
                                 'Sets',                'Set time in min',
       'Break time between sets in min',                     'Total time',
                             'Priority',                              nan,
                               'Biceps',                          'Chest',
                              'Triceps',                       'Shoulder',
                           'Abdominals',                     'Upper Back',
                           'Lower Back',                   'Upper Thighs',
                         'Lower Thighs',                         'Calves',
                               'Glutes'],
      dtype='object', name=0)

In [5]:
import test

weekLen, weekNum, prep, minWork, maxWork, pause, minWeek, maxWeek, early, latest, startDay, startMonth, startYear, zone, allExercise, needs = test.prepare_input("ex1.xlsx")
print("weekLen: ", weekLen)
print("weekNum: ", weekNum)
print("prep: ", prep)
print("minWork: ", minWork)
print("maxWork: ", maxWork)
print("pause: ", pause)
print("minWeek: ", minWeek)
print("maxWeek: ", maxWeek)
print("early: ", early)
print("latest: ", latest)
print("startDay: ", startDay)
print("startMonth: ", startMonth)
print("startYear: ", startYear)
print("zone: ", zone)
print(f"allExercise {len(allExercise)}: {allExercise}")
print("needs: ", needs)


weekLen:  7
weekNum:  3
prep:  25
minWork:  45
maxWork:  100
pause:  5
minWeek:  2
maxWeek:  4
early:  8
latest:  22
startDay:  20
startMonth:  7
startYear:  2023
zone:  2
allExercise 46: [{'name': 'Cycling Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': -1, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': -1, 'backLow': -1, 'thighUp': 0, 'thighLow': 0, 'calves': 0, 'glutes': -1}, {'name': 'Rowing Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': 0, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': 0, 'backLow': 0, 'thighUp': -1, 'thighLow': -1, 'calves': -1, 'glutes': -1}, {'name': 'Running Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': -1, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': -1, 'backLow': -1, 'thighUp': 0, 'thighLow': 0, 'calves': 0, 'glutes': 0}, {'name': 'Abduction Machine', 'category': 'resistance', 'setTime': 6, 'priority': 3, 'biceps'

### Write Up on the constraints

The objective is the maximization of the total sum of priorities over all
performed exercises within the optimization time frame

**Workouts**
- Workout routine comprises of one or multiple weeks, choosen by customer
- At max **1 Workout** per day and always in the largest possible free time slot
- 3 Types of exercises: warm up, cardio, resistance, each workout has exactly one warm up
- Between exercises: constant break
- Before and after workout: constant prep time each
- exercises include different body parts: there must be breaks (consecutive days) between exercising body parts
- Resting days for a body part: use maximum value of exercise

**Data constraint**
- given first day and weeks to optimize
- time frame for each day
- maximum and minimum workouts per week
- miminum workout for each body part (in minutes)
- some exercises don't require resting a body part, however, cannot be executed
if already resting on that body part


**Danger**
- Calendar has past events, must be considered for resting days
- user data is timezone dependent

In [156]:
import dataclasses
import pytz
import datetime
import pandas as pd
import re

@dataclasses.dataclass
class ExPlan: 
    name: str                # exercise name 
    _sets: int                # number of sets
    _perSetTime: int          # time per set in minutes
    _breakTime: int           # break time between sets in minutes
    totalTime: int           # total time of exercise in minutes
    priority: int            # priority of exercise, higher is more important
    biceps: int              # biceps muscle group, days of rest after exercise
    chest: int               # chest muscle group, days of rest after exercise
    shoulder: int            # sholders muscle group, days of rest after exercise
    triceps: int             # triceps muscle group, days of rest after exercise
    abdominals: int          # abdominals muscle group, days of rest after exercise
    upperBack: int           # upperBack muscle group, days of rest after exercise
    lowerBack: int           # lowerBack muscle group, days of rest after exercise
    upperThighs: int         # upperThighs muscle group, days of rest after exercise
    lowerThighs: int         # lowerThighs muscle group, days of rest after exercise
    calves: int              # calves muscle group, days of rest after exercise
    glutes: int             # glutues muscle group, days of rest after exercise

    def parse_body_part(frame: pd.DataFrame, attr: str) -> int:
        if pd.isnull(frame[attr]): # if we have nan value, we say it is -1
            return -1
        if isinstance(frame[attr], int):
            return frame[attr]
        else:
            return -1 # if not any meaningfull data type, we just say it is -1

    def __init__(self, df: pd.DataFrame):
        self.biceps = ExPlan.parse_body_part(df, "Biceps")
        self.chest = ExPlan.parse_body_part(df, "Chest")
        self.shoulder = ExPlan.parse_body_part(df, "Shoulder")
        self.triceps = ExPlan.parse_body_part(df, "Triceps")
        self.abdominals = ExPlan.parse_body_part(df, "Abdominals")
        self.upperBack = ExPlan.parse_body_part(df, "Upper Back")
        self.lowerBack = ExPlan.parse_body_part(df, "Lower Back")
        self.upperThighs = ExPlan.parse_body_part(df, "Upper Thighs")
        self.lowerThighs = ExPlan.parse_body_part(df, "Lower Thighs")
        self.calves = ExPlan.parse_body_part(df, "Calves")
        self.glutes = ExPlan.parse_body_part(df, "Glutes")
        self.name = df["Name"]
        self.totalTime = df["Total time"] # Todo: sanity check against other time values
        self.priority = df["Priority"]
        self._sets = -1 # Todo: sanity check against other time values
        self._perSetTime = -1 
        self._breakTime = -1


    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"ExPlan(\n\tname={self.name},\n\t_sets={self._sets},\n\t_perSetTime={self._perSetTime},\n\t_breakTime={self._breakTime},\n\ttotalTime={self.totalTime},\n\tpriority={self.priority},\n\tbiceps={self.biceps},\n\tchest={self.chest},\n\tshoulder={self.shoulder},\n\ttriceps={self.triceps},\n\tabdominals={self.abdominals},\n\tupperBack={self.upperBack},\n\tlowerBack={self.lowerBack},\n\tupperThighs={self.upperThighs},\n\tlowerThighs={self.lowerThighs},\n\tcalves={self.calves},\n\tglutes={self.glutes}\n)"

@dataclasses.dataclass
class NeedsPlan:
    biceps: int              # biceps muscle group, days of rest after exercise
    chest: int               # chest muscle group, days of rest after exercise
    shoulder: int            # sholders muscle group, days of rest after exercise
    triceps: int             # triceps muscle group, days of rest after exercise
    abdominals: int          # abdominals muscle group, days of rest after exercise
    upperBack: int           # upperBack muscle group, days of rest after exercise
    lowerBack: int           # lowerBack muscle group, days of rest after exercise
    upperThighs: int         # upperThighs muscle group, days of rest after exercise
    lowerThighs: int         # lowerThighs muscle group, days of rest after exercise
    calves: int              # calves muscle group, days of rest after exercise
    glutes: int             # glutues muscle group, days of rest after exercise

    def parse_value(frame: pd.DataFrame, attr: str) -> int:
        if pd.isnull(frame[attr]): # if we have nan value, we just say it is 0
            return 0
        if isinstance(frame[attr], int):
            return frame[attr]
        else:
            return 0 # if not any meaningfull data type, we just say it is 0
    # a single row!
    def __init__(self, exercise_dataset: pd.DataFrame):
        self.biceps = NeedsPlan.parse_value(exercise_dataset, "Biceps")
        self.chest = NeedsPlan.parse_value(exercise_dataset, "Chest")
        self.shoulder = NeedsPlan.parse_value(exercise_dataset, "Shoulder")
        self.triceps = NeedsPlan.parse_value(exercise_dataset, "Triceps")
        self.abdominals = NeedsPlan.parse_value(exercise_dataset, "Abdominals")
        self.upperBack = NeedsPlan.parse_value(exercise_dataset, "Upper Back")
        self.lowerBack = NeedsPlan.parse_value(exercise_dataset, "Lower Back")
        self.upperThighs = NeedsPlan.parse_value(exercise_dataset, "Upper Thighs")
        self.lowerThighs = NeedsPlan.parse_value(exercise_dataset, "Lower Thighs")
        self.calves = NeedsPlan.parse_value(exercise_dataset, "Calves")
        self.glutes = NeedsPlan.parse_value(exercise_dataset, "Glutes")


    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"NeedsPlan(\n\tbiceps={self.biceps},\n\tchest={self.chest},\n\tshoulder={self.shoulder},\n\ttriceps={self.triceps},\n\tabdominals={self.abdominals},\n\tupperBack={self.upperBack},\n\tlowerBack={self.lowerBack},\n\tupperThighs={self.upperThighs},\n\tlowerThighs={self.lowerThighs},\n\tcalves={self.calves},\n\tglutes={self.glutes}\n)"


@dataclasses.dataclass
class ExInfo: # describes the full exercise plan
    weekLen: int            # length of a week in days
    weekNum: int            # number of weeks to plan 
    prep: int               # preparation time in minutes, before and after workout
    minWork: int            # minimum time a workout, not including prep (before and after), including pause
    maxWork: int            # maximum time a workout, not including prep (before and after), including pause
    pause: int              # pause between workouts in minutes
    minWeek: int            # minimum number of workouts per week
    maxWeek: int            # maximum number of workouts per week
    early: int              # earliest hour of day to start a workout (inclusive prep)
    latest: int             # latest hour of day to start a workout (inclusive prep)
    startDay: datetime.date # day to start the plan
    zone: datetime.timezone # timezone of the plan
    allExercise: dict[str, list[ExPlan]]   # all exercises to plan
    needs: NeedsPlan        # needs to play the plan


    def __init__(self, ex_path):
        info_dataset = pd.read_excel(ex_path, sheet_name="info")
        self.sanity_check_info(info_dataset)
        exercise_dataset = pd.read_excel(ex_path, sheet_name="exercises")

        exercise_dataset.columns = exercise_dataset.iloc[0] # load first row as column names
        exercise_dataset = exercise_dataset.drop(exercise_dataset.index[0]) # drop first row

        
        self.allExercise = {}
        self.allExercise["warmup"] = []
        self.allExercise["resistance"] = []
        self.allExercise["cardio"] = []
        self.needs = None
        self.sanity_check_needs(exercise_dataset)

        exercise_dataset = exercise_dataset.drop(exercise_dataset.index[-1])
        self.sanity_check_exercise(exercise_dataset)


    def sanity_check_needs(self, exercise_dataset: pd.DataFrame):
        last_row = exercise_dataset.iloc[-1]
        assert last_row["Name"] == "Minimum Weekly Workout Time", "last row of exercise dataset is not minimum weekly workout time"
        self.needs = NeedsPlan(last_row)
        # drop last row
        exercise_dataset = exercise_dataset.drop(exercise_dataset.index[-1])

    def sanity_check_exercise(self, exercise_dataset: pd.DataFrame):
        for index, row in exercise_dataset.iterrows():
            category: str = row["Category"]
            if pd.isnull(category):
                continue
            self.allExercise[category.lower().replace(" ", "")].append(ExPlan(row))
        
    def sanity_check_info(self, info_dataset: pd.DataFrame):
        assert info_dataset.shape == (12, 2), "info dataset has unexpected shape"
        # A table of Parameter: Value, we are guaranteed that the parameters stay the same

        assert isinstance(info_dataset.iloc[0, 1], int), "weekLen is not an int"
        assert info_dataset.iloc[0, 1] >= 0, f"unexpected weekLen of {info_dataset.iloc[0, 1]}"
        assert info_dataset.iloc[0, 1] <= 7, f"unexpected weekLen of {info_dataset.iloc[0, 1]}"
        self.weekLen = info_dataset.iloc[0, 1]
        assert isinstance(info_dataset.iloc[1, 1], int), "weekNum is not an int"
        assert info_dataset.iloc[1, 1] >= 0, f"unexpected weekNum of {info_dataset.iloc[1, 1]}"
        self.weekNum = info_dataset.iloc[1, 1]
        assert isinstance(info_dataset.iloc[2, 1], int), "prep is not an int"
        assert info_dataset.iloc[2, 1] >= 0, f"unexpected prep of {info_dataset.iloc[2, 1]}"
        self.prep = info_dataset.iloc[2, 1]
        assert isinstance(info_dataset.iloc[3, 1], int), "minWork is not an int"
        assert info_dataset.iloc[3, 1] >= 0, f"unexpected minWork of {info_dataset.iloc[3, 1]}"
        self.minWork = info_dataset.iloc[3, 1]
        assert isinstance(info_dataset.iloc[4, 1], int), "maxWork is not an int"
        assert info_dataset.iloc[4, 1] >= 0, f"unexpected maxWork of {info_dataset.iloc[4, 1]}"
        assert info_dataset.iloc[4, 1] >= self.minWork, f"maxWork {info_dataset.iloc[4, 1]} smaller than minWork {self.minWork}"
        self.maxWork = info_dataset.iloc[4, 1]
        assert isinstance(info_dataset.iloc[5, 1], int), "pause is not an int"
        assert info_dataset.iloc[5, 1] >= 0, f"unexpected pause of {info_dataset.iloc[5, 1]}"
        self.pause = info_dataset.iloc[5, 1]
        assert isinstance(info_dataset.iloc[6, 1], int), "minWeek is not an int"
        assert info_dataset.iloc[6, 1] >= 0, f"unexpected minWeek of {info_dataset.iloc[6, 1]}"
        self.minWeek = info_dataset.iloc[6, 1]
        assert isinstance(info_dataset.iloc[7, 1], int), "maxWeek is not an int"
        assert info_dataset.iloc[7, 1] >= 0, f"unexpected maxWeek of {info_dataset.iloc[7, 1]}"
        assert info_dataset.iloc[7, 1] >= self.minWeek, f"maxWeek {info_dataset.iloc[7, 1]} smaller than minWeek {self.minWeek}"
        self.maxWeek = info_dataset.iloc[7, 1]
        assert isinstance(info_dataset.iloc[8, 1], int), "early is not an int"
        assert info_dataset.iloc[8, 1] >= 0, f"unexpected early of {info_dataset.iloc[8, 1]}"
        assert info_dataset.iloc[8, 1] <= 24, f"unexpected early of {info_dataset.iloc[8, 1]}"
        self.early = info_dataset.iloc[8, 1]
        assert isinstance(info_dataset.iloc[9, 1], int), "latest is not an int"
        assert info_dataset.iloc[9, 1] >= 0, f"unexpected latest of {info_dataset.iloc[9, 1]}"
        assert info_dataset.iloc[9, 1] <= 24, f"unexpected latest of {info_dataset.iloc[9, 1]}"
        assert info_dataset.iloc[9, 1] >= self.early, f"latest {info_dataset.iloc[9, 1]} smaller than early {self.early}"
        self.latest = info_dataset.iloc[9, 1]
        assert isinstance(info_dataset.iloc[10, 1], datetime.date), "startDay is not a date"
        assert isinstance(info_dataset.iloc[11, 1], str), "zone is not a string"
        # remove spaces from zone
        zone = info_dataset.iloc[11, 1].replace(" ", "")
        # assuming UTC+X format
        offset_hours = int(re.search(r'\d+', zone).group())
        tz = datetime.timezone(datetime.timedelta(hours=offset_hours))
        self.startDay = info_dataset.iloc[10, 1]
        self.zone = tz
    
    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"ExInfo(\n\tweekLen={self.weekLen},\n\tweekNum={self.weekNum},\n\tprep={self.prep},\n\tminWork={self.minWork},\n\tmaxWork={self.maxWork},\n\tpause={self.pause},\n\tminWeek={self.minWeek},\n\tmaxWeek={self.maxWeek},\n\tearly={self.early},\n\tlatest={self.latest},\n\tstartDay={self.startDay},\n\tzone={self.zone},\n\tallExercise={self.allExercise},\n\tneeds={self.needs}\n)"



In [209]:
import icalendar
import dataclasses
import datetime
import pytz

@dataclasses.dataclass
class UserEvent:
    event: str
    isExercise: bool
    exercises: list[str]
    start: datetime.datetime
    end: datetime.datetime
    stamp: datetime.datetime
    def __init__(self, event, start, end, stamp, isExercise=False, exercises=[]):
        self.event = event
        self.start = start
        self.end = end
        self.stamp = stamp
        self.isExercise = isExercise
        self.exercises = exercises
    
    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"UserEvent(\n\tevent={self.event},\n\tisExercise={self.isExercise},\n\texercises={self.exercises},\n\tstart={self.start},\n\tend={self.end},\n\tstamp={self.stamp}\n)"

@dataclasses.dataclass
class UserCalendar: 
    events: list[UserEvent] 
    tz_shift: datetime.timezone
    def __init__(self, tz):
        self.tz_shift = tz
        self.events = []

    # note: Fixed timezones!
    def load_calendar(self, cpath):
        file = open(cpath)
        cal = icalendar.Calendar.from_ical(file.read())
        for component in cal.walk():
            if component.name == "VEVENT":
                ds: datetime.datetime = component.decoded("dtstart")
                dt: datetime.datetime = component.decoded("dtend")
                dst: datetime.datetime = component.decoded("dtstamp")
                ds = ds.replace(tzinfo=datetime.timezone.utc).astimezone(tz=self.tz_shift)
                dt = dt.replace(tzinfo=datetime.timezone.utc).astimezone(tz=self.tz_shift)
                dst= dst.replace(tzinfo=datetime.timezone.utc).astimezone(tz=self.tz_shift)
                event: str = str(component.decoded("summary").decode("utf-8") )
                event = event.lower().replace(" ","")
                if event.startswith("workout"):
                    desc = component.get("description")
                    desc = [x.strip() for x in desc.split(',')[0].split("\n")]
                    self.events.append(UserEvent(
                        event = event,
                        start = ds,
                        end = dt, 
                        stamp = dst,
                        isExercise = True,
                        exercises = desc
                    ))
                else:
                    self.events.append(UserEvent(
                        event = event,
                        start = ds,
                        end = dt, 
                        stamp = dst
                    ))
                    
        file.close()
    
    def __repr__(self) -> str:
        # pretty print the class with line breaks and attribute names
        return f"UserCalendar(\n\tevents={self.events},\n\ttz_shift={self.tz_shift}\n)"

c = UserCalendar(datetime.timezone(datetime.timedelta(hours=2)))
c.load_calendar("cal1.ics")
print(c)

UserCalendar(
	events=[UserEvent(
	event=heuristicoptimization:exercise,
	isExercise=False,
	exercises=[],
	start=2023-07-13 08:30:00+02:00,
	end=2023-07-13 10:00:00+02:00,
	stamp=2023-07-13 08:30:00+02:00
), UserEvent(
	event=heuristicoptimization:lecture,
	isExercise=False,
	exercises=[],
	start=2023-07-13 10:00:00+02:00,
	end=2023-07-13 11:30:00+02:00,
	stamp=2023-07-13 10:00:00+02:00
), UserEvent(
	event=columngenerationundbranch-and-price:lecture,
	isExercise=False,
	exercises=[],
	start=2023-07-13 11:30:00+02:00,
	end=2023-07-13 13:00:00+02:00,
	stamp=2023-07-13 11:30:00+02:00
), UserEvent(
	event=lunchbreak,
	isExercise=False,
	exercises=[],
	start=2023-07-13 13:00:00+02:00,
	end=2023-07-13 14:30:00+02:00,
	stamp=2023-07-13 13:00:00+02:00
), UserEvent(
	event=studyinginlibrary,
	isExercise=False,
	exercises=[],
	start=2023-07-13 14:30:00+02:00,
	end=2023-07-13 17:30:00+02:00,
	stamp=2023-07-13 14:30:00+02:00
), UserEvent(
	event=coffeebreak,
	isExercise=False,
	exercises=[],
	st

In [212]:


def build_model(exercise_path, calender_path):
    ex = ExInfo(exercise_path)
    cal = UserCalendar(ex.zone) 
    cal.load_calendar(calender_path)
    print(cal)
    print(ex)
    
build_model("ex1.xlsx", "cal1.ics")

UserCalendar(
	events=[UserEvent(
	event=heuristicoptimization:exercise,
	isExercise=False,
	exercises=[],
	start=2023-07-13 08:30:00+02:00,
	end=2023-07-13 10:00:00+02:00,
	stamp=2023-07-13 08:30:00+02:00
), UserEvent(
	event=heuristicoptimization:lecture,
	isExercise=False,
	exercises=[],
	start=2023-07-13 10:00:00+02:00,
	end=2023-07-13 11:30:00+02:00,
	stamp=2023-07-13 10:00:00+02:00
), UserEvent(
	event=columngenerationundbranch-and-price:lecture,
	isExercise=False,
	exercises=[],
	start=2023-07-13 11:30:00+02:00,
	end=2023-07-13 13:00:00+02:00,
	stamp=2023-07-13 11:30:00+02:00
), UserEvent(
	event=lunchbreak,
	isExercise=False,
	exercises=[],
	start=2023-07-13 13:00:00+02:00,
	end=2023-07-13 14:30:00+02:00,
	stamp=2023-07-13 13:00:00+02:00
), UserEvent(
	event=studyinginlibrary,
	isExercise=False,
	exercises=[],
	start=2023-07-13 14:30:00+02:00,
	end=2023-07-13 17:30:00+02:00,
	stamp=2023-07-13 14:30:00+02:00
), UserEvent(
	event=coffeebreak,
	isExercise=False,
	exercises=[],
	st

In [4]:
import gurobipy as gp
def solve(full_instance_path, calendar_path):
    pass